In [ ]:
#line only needed when running in google Colab
#!pip install jupyter-dash==0.2.1

In [ ]:
import pandas as pd #
import numpy as np
import seaborn as sns
import matplotlib
%matplotlib inline

import plotly.graph_objects as go
import matplotlib.pyplot as plt

from plotly.subplots import make_subplots
from datetime import date

import plotly
# Import the necessaries libraries to work offline
import plotly.offline as pyo
# Set notebook mode to work in offline
pyo.init_notebook_mode()


from jupyter_dash import JupyterDash
from dash import dcc
#import dash_html_components as html
from dash import html
from dash.dependencies import Input, Output, State

import dash
from dash import dash_table
import time

plotly.__version__

In [ ]:
# loading data from the file
seasons = pd.read_csv('seasons.csv')                               
circuits = pd.read_csv('circuits.csv')                             
status = pd.read_csv('status.csv')                                 
constructors = pd.read_csv('constructors.csv', names=['constructorId', 'constructorRef', 
         'constructorName', 'constructorNationality', 'constructorUrl'],header=0)                     
drivers = pd.read_csv('drivers.csv',names=['driverId', 'driverRef', 'driverNumber', 'driverCode', 
         'driverForename', 'driverSurname', 'driverDob','driverNationality', 'driverUrl'],header=0) 
#races = pd.read_csv('races.csv', names=['raceId', 'raceYear', 'raceRound', 'raceCircuitId', 
#         'raceName', 'raceDate', 'raceTime', 'raceUrl'],header=0)                                   
races = pd.read_csv('races.csv', names=['raceId', 'raceYear', 'raceRound', 'raceCircuitId', 
         'raceName', 'raceDate', 'raceTime', 'raceUrl','N1','N2','N3','N4','N5','N6','N7','N8','N9','N10'],header=0)  
races=races[['raceId', 'raceYear', 'raceRound', 'raceCircuitId', 
         'raceName', 'raceDate', 'raceTime', 'raceUrl']]
pit_stops = pd.read_csv('pit_stops.csv')                           
qualifying = pd.read_csv('qualifying.csv')                         
constructor_results = pd.read_csv('constructor_results.csv')       
constructor_standings = pd.read_csv('constructor_standings.csv')   
results = pd.read_csv('results.csv')                               
driver_standings = pd.read_csv('driver_standings.csv')             
lap_times = pd.read_csv('lap_times.csv')     
sprint_results= pd.read_csv('sprint_results.csv', names=['resultId', 'raceId', 'driverId', 'constructorId', 
        'number', 'sr_grid','position', 'positionText', 'sr_positionOrder', 'sr_points', 'sr_laps', 'sr_time',
       'milliseconds', 'fastestLap', 'fastestLapTime', 'statusId'],header=0) 
sprint_results=sprint_results[['raceId','driverId','sr_grid','sr_laps','sr_time','sr_points']]

results = results.rename(columns={'rank': 'fastestLapRank'})
results

In [ ]:
#to create a new dataframe "df" merging parts of: results + races + drivers + constructors
df= pd.merge(results, races, on='raceId')
df= pd.merge(df, drivers, on='driverId')
df= pd.merge(df, constructors, on='constructorId')
df= pd.merge(df, sprint_results, on=['raceId','driverId'],how='left')
df['sr_points'] = df['sr_points'].fillna(0)
df['points']=df['points']+df['sr_points']

df = df.drop(['resultId', 'number','position', 'positionText', #'time',
              'raceUrl',
              'driverRef','driverCode','driverUrl',
              'constructorRef','constructorUrl'], axis = 1)

#to get the driver age at the date of the race
df['raceDate']=pd.to_datetime(df['raceDate'])
df['driverDob']=pd.to_datetime(df['driverDob'])
df['driverAge']=((df['raceDate'] - df['driverDob'])/ np.timedelta64(1, 'Y')).round(2)

df['driverName']=df['driverForename']+" "+df['driverSurname']

#to reformat the date
df['raceDate']=df['raceDate'].dt.strftime("%Y-%m-%d")
#df['driverDob']=df['driverDob'].dt.strftime("%Y-%m-%d")


#to reorganize the columns and remove: raceCircuitId & constructorId
df=df[['raceId','raceYear', 'raceRound', 'raceName', 'raceDate', 'raceTime',
       'driverId','driverNumber','driverName', 'driverDob', 'driverNationality', 'driverAge',
       'constructorName', 'constructorNationality',
       'grid', 'positionOrder',
       'points', 'laps', 'time','milliseconds', 'fastestLap', 'fastestLapRank',
       'fastestLapTime', 'fastestLapSpeed', 'statusId',
       'sr_grid','sr_laps','sr_time','sr_points']]


lastqualiId=qualifying['raceId'][qualifying['raceId']==qualifying['raceId'].max()].iloc[0]
lastqualyrace=races['raceDate'][races['raceId']==lastqualiId].values[0]
print(f"Last Qualification: {lastqualyrace}")
lastrace=df['raceDate'].max()
print(f"Last Race: {lastrace}")
df[df['raceDate']==lastrace].sort_values('positionOrder')
df

In [ ]:
###########################
####### For driver Tabs
###########################
df0= df.groupby(['raceYear','driverName','driverNationality','driverDob','driverId','constructorName','constructorNationality'])['points'].sum().reset_index().sort_values(
                                    ['raceYear','points'], ascending=[False,False]).groupby('raceYear').head(3)
df0['count'] = df0['points'].ne(df0['raceYear'].shift(1)).cumsum()

#to identify the winning drivers
df0['place']= df0['count']-df0['raceYear'].apply(lambda y:df0[df0["raceYear"]==y].min()["count"])+1
df0=df0.drop(['count'], axis = 1)
df0['raceYear_place']=df0['raceYear'].astype(str)+"_"+df0['place'].astype(str)

mymap={1:'#e10600',2:'#e3a19f',3:'#e3dcdc'}
df0['color']=df0['place'].map(mymap)
df0=df0.sort_values(by = 'raceYear', ascending=True)

####### For Charts Place per Season
#to get the data frame with Driver's place per season
dfp = df.groupby(['raceYear','driverName'])['points'].sum().reset_index().sort_values(
                                    ['raceYear','points'], ascending=[False,False]).groupby('raceYear').head(20)
dfp['count'] = dfp['points'].ne(dfp['raceYear'].shift(1)).cumsum()
dfp['place']= dfp['count']-dfp['raceYear'].apply(lambda y:dfp[dfp["raceYear"]==y].min()["count"])+1
dfp8=dfp.drop(['count'], axis = 1)

####### For Chart Constructors per Season
#to create a dataframe with a different random color per driver
dfcd=df.groupby(['driverName'])['points'].sum().reset_index()
dfcd['color']=np.random.uniform(0.8,1,len(dfcd))
dfcd=dfcd[['driverName','color']]

#to get the data frame with Driver's place per season
dfp = df.groupby(['raceYear','driverName','constructorName'])['points'].sum().reset_index().sort_values(
                                    ['raceYear','points'], ascending=[False,False]).groupby('raceYear').head(20)
dfp['count'] = dfp['points'].ne(dfp['raceYear'].shift(1)).cumsum()
dfp['place']= dfp['count']-dfp['raceYear'].apply(lambda y:dfp[dfp["raceYear"]==y].min()["count"])+1
dfp10=dfp.drop(['count'], axis = 1)

####### For Chart Youngest Drivers
dfy=df.copy()
dfy['raceDate']=pd.to_datetime(dfy['raceDate'])


###########################
####### For Constructor Tabs
###########################
df0c= df.groupby(['raceYear','constructorName','constructorNationality'])['points'].sum().reset_index().sort_values(
                                    ['raceYear','points'], ascending=[False,False]).groupby('raceYear').head(3)
df0c['count'] = df0c['points'].ne(df0c['raceYear'].shift(1)).cumsum()

#to identify the winning drivers
df0c['place']= df0c['count']-df0c['raceYear'].apply(lambda y:df0c[df0c["raceYear"]==y].min()["count"])+1
df0c=df0c.drop(['count'], axis = 1)
df0c['raceYear_place']=df0c['raceYear'].astype(str)+"_"+df0c['place'].astype(str)

mymap={1:'#e10600',2:'#e3a19f',3:'#e3dcdc'}
df0c['color']=df0c['place'].map(mymap)
df0c=df0c.sort_values(by = 'raceYear', ascending=True)

####### For Charts Place per Season
#to get the data frame with Constructor's place per season
dfp = df.groupby(['raceYear','constructorName'])['points'].sum().reset_index().sort_values(
                                    ['raceYear','points'], ascending=[False,False]).groupby('raceYear').head(20)
dfp['count'] = dfp['points'].ne(dfp['raceYear'].shift(1)).cumsum()
dfp['place']= dfp['count']-dfp['raceYear'].apply(lambda y:dfp[dfp["raceYear"]==y].min()["count"])+1
dfp9=dfp.drop(['count'], axis = 1)

####### For Chart Drivers per Season
#to create a dataframe with a different random color per constructor
dfcc=df.groupby(['constructorName'])['points'].sum().reset_index()
dfcc['color']=np.random.uniform(0.8,1,len(dfcc))
dfcc=dfcc[['constructorName','color']]

#to get the data frame with Driver's place per season
dfp = df.groupby(['raceYear','driverName','constructorName'])['points'].sum().reset_index().sort_values(
                                    ['raceYear','points'], ascending=[False,False]).groupby('raceYear').head(20)
dfp['count'] = dfp['points'].ne(dfp['raceYear'].shift(1)).cumsum()
dfp['place']= dfp['count']-dfp['raceYear'].apply(lambda y:dfp[dfp["raceYear"]==y].min()["count"])+1
dfp11=dfp.drop(['count'], axis = 1)


###########################
####### for 'Formula 1 Dashboard'
###########################

years = [{'label': 'All', 'value': 'All'}]
for year in df['raceYear'].sort_values(ascending=False).unique():
    years.append({'label':str(year),'value':year})
del years[0]        

topics = [{'label':'RACES','value':'RACES'}]
topics.append({'label':'DRIVERS','value':'DRIVERS'})
topics.append({'label':'TEAMS','value':'TEAMS'})
topics.append({'label':'FASTEST LAP','value':'FASTEST LAP'})


def create_table(iD):
    
    return dash_table.DataTable(id=iD, 
                                 style_cell={'textAlign': 'left','border': 'none', 'color': '#808080'}, 
                                 style_header={
                                            'backgroundColor': 'white',
                                            'fontWeight': 'bold',
                                            'font-size': '16px',
                                        },
                                 style_data_conditional=[
                                        {
                                            'if': {'row_index': 'even'},
                                            'backgroundColor': '#f4f4f4',
                                        }]
                                    )

def cumtimepits(df, driver, stop):
    time = df['TIME'][(df['STOPS']<=stop)&(df['DRIVER']==driver)].sum()
    return time

def startingtime(q1,q2,q3):
    if q3 != "\\N":   time = q3
    elif q2 != "\\N": time = q2
    elif q1 != "\\N": time = q1
    else:             time = "\\N"
    return time


In [ ]:
app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Checklist(
        ['New York City', 'Montréal', 'San Francisco'],
        ['New York City', 'Montréal'],
        labelStyle={'display': 'block'},
    )
])

app.run_server(mode='inline', port=8077)

In [ ]:
tickfont=12
barsColor = "#f55b68"
barsColor = '#385d7f'
barsColor = '#e10600'
style_dropdown= {'font-size': '14px', 'color':'#808080','font-family':"system-ui",}

rectangle = html.Div([
        html.Div([
                html.P(
                    "This Dashboard was made purely in Python using the Dash library from Plotly & with tons of love ♡",
                    style={'fontSize': 14},
                    className="row",
                )
            ],className="legend",)
    ],style={'width':'100%','display':'inline-block'})


app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div([
        html.Div([html.Img(src=app.get_asset_url("f1_logo.jpg"),className='logo')],
                  style= {'width': '8%', 'display': 'inline-block'}),
        html.Div([html.H3("Miguel's Formula 1 Dashboard",className='dashboard-name')],
                 style= {'width': '92%', 'display': 'inline-block'}),
    ],className='dashboard-name'),
    rectangle,
    html.Br([],style= {'color': '#ffffff'}),
    html.Div([
    html.Div(id="mainTabsDiv",
            children=[
                    dcc.Tabs(id="mainTabs",
                        value = "Driver Results",
                        children=[
                        dcc.Tab(label='Driver Results', value='Driver Results',className='main-tab',selected_className='main-tab--selected'),
                        dcc.Tab(label='Constructor Results', value='Constructor Results',className='main-tab',selected_className='main-tab--selected'),
                        #dcc.Tab(label='Circuit Results', value='Circuit Results',className='main-tab',selected_className='main-tab--selected'),
                        dcc.Tab(label='Results per Nationality', value='Nationality Results',className='main-tab',selected_className='main-tab--selected'),
                        dcc.Tab(label='Formula 1 Dashboard', value='Formula 1 Dashboard',className='main-tab',selected_className='main-tab--selected'),
                    ])
            ]
    ),
    html.Br([]),
    html.Div([
                dcc.Tabs(id="secondaryTabs",vertical=True, className='custom-tabs')
            ],style= {'width': '18%', 'display': 'inline-block', 'vertical-align': 'top'}),
    html.Div([
        html.Div(id='filters', children=[

            #####filters_podiums
            html.Div(id="filters_podiums", children=[
                html.Div([
                    html.H5('Observations:',
                          style= {'color':'#808080','font-weight':"bold", 'width': '60%', 'display': 'inline-block'}),
                    html.Div([
                        dcc.RadioItems(
                            id='RadioItems',
                            options=[
                                {'label': '10', 'value': 10},
                                {'label': '20', 'value': 20},
                                {'label': '30', 'value': 30},
                            ],
                            value=10,
                            labelStyle = dict(display='block', color='#808080'),
                            style= {'display': 'inline-block'}
                        ),
                    ],style= {'width': '40%', 'display': 'inline-block',}),###################
                ],style= {'width': '17%', 'display': 'inline-block'}),
                
                html.Div([
                    html.H5('Select places:',
                          style= {'color':'#808080','font-weight':"bold",'width': '60%', 'display': 'inline-block'}),
                    dcc.Checklist(
                        id='Checklist',
                        options=[
                            {'label': '1st', 'value': '1'},
                            {'label': '2nd', 'value': '2'},
                            {'label': '3rd', 'value': '3'},
                        ],
                        value=['1','2','3'],
                        labelStyle = dict(display='block', color='#808080'),
                        style= {'width': '40%', 'display': 'inline-block'}
                    ),
                ],style= {'width': '17%', 'display': 'inline-block'}),
                
                html.Div(id="filters_nationality", children=[
                    html.H5('Select Nationality:',
                          style= {'color':'#808080','font-weight':"bold", 'width': '40%', 'display': 'inline-block'}),
                    html.Div([
                        dcc.Dropdown(
                            id='Dropdown_Nationalities',
                            value='All',
                            style= style_dropdown)
                    ],style= {'width': '55%', 'display': 'inline-block',}),###################
                ],style= {'width': '30%', 'display': 'inline-block'}),
            ],style= {'width': '100%', 'display': 'inline-block'}),

            ######filters_placePerSeasonDrivers
            html.Div(id='filters_placePerSeasonDrivers', children=[
                html.Div([
                    html.H5('Select a Driver:',
                          style= {'color':'#808080','font-weight':"bold", 'width': '33%', 'display': 'inline-block'}),
                    html.Div([
                        dcc.Dropdown(
                            id='DropdownDriver1',
                            value='Lewis Hamilton',
                            style= style_dropdown)
                    ],style= {'width': '60%', 'display': 'inline-block',}),###################
                ],style= {'width': '32%', 'display': 'inline-block'}),
                html.Div([
                    html.H5('Select a Driver:',
                          style= {'color':'#808080','font-weight':"bold", 'width': '33%', 'display': 'inline-block'}),
                    html.Div([
                        dcc.Dropdown(
                            id='DropdownDriver2',
                            value='Sergio Pérez',
                            style= style_dropdown)
                    ],style= {'width': '60%', 'display': 'inline-block',}),###################
                ],style= {'width': '32%', 'display': 'inline-block'}),
            ],style= {'display': 'None'}),

            #####filters_placePerSeasonConstructors
            html.Div(id='filters_placePerSeasonConstructors', children=[
                html.Div([
                    html.H5('Select a Constructor:',
                          style= {'color':'#808080','font-weight':"bold", 'width': '33%', 'display': 'inline-block'}),
                    html.Div([
                        dcc.Dropdown(
                            id='DropdownConstructor1',
                            value='Mercedes',
                            style= style_dropdown)
                    ],style= {'width': '60%', 'display': 'inline-block',}),###################
                ],style= {'width': '50%', 'display': 'inline-block'}),
                html.Div([
                    html.H5('Select a Constructor:',
                          style= {'color':'#808080','font-weight':"bold", 'width': '33%', 'display': 'inline-block'}),
                    html.Div([
                        dcc.Dropdown(
                            id='DropdownConstructor2',
                            value='Red Bull',
                            style= style_dropdown)
                    ],style= {'width': '60%', 'display': 'inline-block',}),###################
                ],style= {'width': '50%', 'display': 'inline-block'}),
            ],style= {'display': 'None'}),

            #####filters_constructorsPerSeason
            html.Div(id='filters_constructorsPerSeason', children=[
                html.Div([
                    html.H5('Select a Driver:',
                          style= {'color':'#808080','font-weight':"bold", 'width': '33%', 'display': 'inline-block'}),
                    html.Div([
                        dcc.Dropdown(
                            id='DropdownDriverCPS',
                            value='Sergio Pérez',
                            style= style_dropdown)
                    ],style= {'width': '60%', 'display': 'inline-block',}),###################
                ],style= {'width': '40%', 'display': 'inline-block'}),
                html.Div([
                    html.H5('Years to consider:',
                              style= {'color':'#808080','font-weight':"bold", 'width': '15%', 'display': 'inline-block'}),
                    html.Div(dcc.RangeSlider(
                        id='RangeSliderCPS',
                        min=df['raceYear'].min(),
                        max=df['raceYear'].max(),
                        value=[df['raceYear'].min(),df['raceYear'].max()],
                        tooltip={"placement": "top", "always_visible": True},
                        marks={
                            1950: '1950',
                            1960: '1960',
                            1970: '1970',
                            1980: '1980',
                            1990: '1990',
                            2000: '2000',
                            2010: '2010',
                            2020: '2020',
                        },
                        step=1,
                    ), style={'width': '85%','display': 'inline-block'}),
                ],style= {'width': '60%', 'display': 'inline-block'})
            ],style= {'width': '100%','display': 'None'}),

            #####filters_driversPerSeason
            html.Div(id='filters_driversPerSeason', children=[
                html.Div([
                    html.H5('Select a Constructor:',
                          style= {'color':'#808080','font-weight':"bold", 'width': '33%', 'display': 'inline-block'}),
                    html.Div([
                        dcc.Dropdown(
                            id='DropdownConstructorDPS',
                            value='Mercedes',
                            style= style_dropdown)
                    ],style= {'width': '60%', 'display': 'inline-block',}),###################
                ],style= {'width': '40%', 'display': 'inline-block'}),
                html.Div([
                    html.H5('Years to consider:',
                              style= {'color':'#808080','font-weight':"bold", 'width': '15%', 'display': 'inline-block'}),
                    html.Div(dcc.RangeSlider(
                        id='RangeSliderDPS',
                        min=df['raceYear'].min(),
                        max=df['raceYear'].max(),
                        value=[df['raceYear'].min(),df['raceYear'].max()],
                        tooltip={"placement": "top", "always_visible": True},
                        marks={
                            1950: '1950',
                            1960: '1960',
                            1970: '1970',
                            1980: '1980',
                            1990: '1990',
                            2000: '2000',
                            2010: '2010',
                            2020: '2020',
                        },
                        step=1,
                    ), style={'width': '85%','display': 'inline-block'}),
                ],style= {'width': '60%', 'display': 'inline-block'})
            ],style= {'width': '100%','display': 'None'}),


            #####RangeSliderYears
            html.Div(id="RangeSliderDiv", children=[
                html.H5('Years to consider:',
                          style= {'color':'#808080','font-weight':"bold", 'width': '15%', 'display': 'inline-block'}),
                html.Div(dcc.RangeSlider(
                    id='RangeSlider',
                    min=df['raceYear'].min(),
                    max=df['raceYear'].max(),
                    value=[df['raceYear'].min(),df['raceYear'].max()],
                    tooltip={"placement": "top", "always_visible": True},
                    marks={
                        1950: '1950',
                        1960: '1960',
                        1970: '1970',
                        1980: '1980',
                        1990: '1990',
                        2000: '2000',
                        2010: '2010',
                        2020: '2020',
                    },
                    step=1,
                ), style={'width': '49%','display': 'inline-block'}),
                html.Div(id="filters_circuit", children=[
                    html.H5('Select Circuit:',
                          style= {'color':'#808080','font-weight':"bold", 'width': '25%', 'display': 'inline-block'}),
                    html.Div([
                        dcc.Dropdown(
                            id='Dropdown_Circuits',
                            value='All',
                            style= style_dropdown)
                    ],style= {'width': '75%', 'display': 'inline-block',}),###################
                ],style= {'width': '36%', 'display': 'inline-block'}),
            ],style= {'width': '100%', 'display': 'inline-block'}),

        ], style={
            'width': '100%', 'display': 'inline-block',
            #'borderBottom': 'thin lightgrey solid',
            'backgroundColor': 'rgb(250, 250, 250)',
            'padding': '0px 5px',
        }), # End of id='filters'
        html.Div([
            dcc.Graph(id='Chart'),
            dcc.Graph(id='Chart2'),
            dcc.Markdown(id='nodata', style  = {'color':'#808080','font-weight':"bold"})
        ])        
    ],style= {'width': '82%', 'display': 'inline-block'}),

    html.Div(id='tables', children=[
                html.Div([
                    html.Div([
                        html.H4('Select a year:',
                              style= {'color':'#ffffff','width': '33%', 'display': 'inline-block'}),
                        html.Div([
                            dcc.Dropdown(
                                id='Dropdown1',
                                options=years,
                                value=2022,
                                style= style_dropdown)
                        ],style= {'width': '60%', 'display': 'inline-block',}),###################
                    ],style= {'width': '33%', 'display': 'inline-block'}),
                    html.Div([
                        html.H4('Select Topic:',
                              style= {'color':'#ffffff', 'width': '33%', 'display': 'inline-block'}),
                        html.Div([
                            dcc.Dropdown(
                                id='Dropdown2',
                                options=topics,
                                value='RACES',
                                style= style_dropdown)
                        ],style= {'width': '60%', 'display': 'inline-block',}),###################
                    ],style= {'width': '33%', 'display': 'inline-block'}),
                    html.Div([
                        html.H4('Select:',
                              style= {'color':'#ffffff', 'width': '33%', 'display': 'inline-block'}),
                        html.Div([
                            dcc.Dropdown(
                                id='Dropdown3',
                                value='All',
                                style= style_dropdown)
                        ],style= {'width': '60%', 'display': 'inline-block',}),###################
                    ],style= {'width': '33%', 'display': 'inline-block'}),
                ], style={
                    #'borderBottom': 'thin lightgrey solid',
                    'padding-top': 25,
                    'padding-left': 25,
                    'backgroundColor': '#fa6b6b',

                }),


                html.Br([]),
                html.H2(id="headtext",style= {'color':'#808080','font-weight':"bold"}),
                html.Br([]),
                
                html.Div(id="maintable", children=[create_table("dashtable")]),
                html.Div([
                    html.Div(id="raceTabsDiv",
                            children=[
                                    dcc.Tabs(id="raceTabs",
                                        value = "RACE RESULT",
                                        children=[
                                        dcc.Tab(label='RACE RESULT', value='RACE RESULT',className='custom-tab',selected_className='custom-tab--selected'),
                                        dcc.Tab(label='FASTEST LAPS', value='FASTEST LAPS',className='custom-tab',selected_className='custom-tab--selected'),
                                        dcc.Tab(label='PIT STOP SUMMARY', value='PIT STOP SUMMARY',className='custom-tab',selected_className='custom-tab--selected'),
                                        dcc.Tab(label='STARTING GRID', value='STARTING GRID',className='custom-tab',selected_className='custom-tab--selected'),
                                        dcc.Tab(label='QUALIFYING', value='QUALIFYING',className='custom-tab',selected_className='custom-tab--selected'),
                                        #dcc.Tab(label='PRACTICE 3', value='PRACTICE 3',className='custom-tab'),
                                        #dcc.Tab(label='PRACTICE 2', value='PRACTICE 2',className='custom-tab'),
                                        #dcc.Tab(label='PRACTICE 1', value='PRACTICE 1',className='custom-tab')
                                    ],vertical=True)
                            ],style= {'display': 'none'}
                    )],style= {'width': '12%', 'display': 'inline-block'}),
                html.Div([],style= {'width': '5%', 'display': 'inline-block'}),
                html.Div([
                    html.Div(id="maintable2", children=[create_table("dashtable2")],style= {'display': 'none'}),
                    html.Div(id="Chart3Div", children=[dcc.Graph(id='Chart3'),],style= {'display': 'none'}),
                ],style= {'width': '83%', 'display': 'inline-block'}),
            ], style= {'width': '100%','display': 'None'}),

    ],className="cuerpo"),
    html.Br([]),
    html.Br([]),
    html.Div([
        html.Div([
            html.P("Data source: http://ergast.com/downloads", 
                         style={'fontSize': 12},
                         className="row"),
            html.P("Code Github: https://github.com/MiguelG26/Public_Projects/tree/master/db-Formula1%20app", 
                         style={'fontSize': 12},
                         className="row")
        ],className='legend2'),
    ],style={'width':'100%','display':'inline-block'}),
])


@app.callback(
     [Output('secondaryTabs', 'children'),
      Output('secondaryTabs', 'value')],
     Input('mainTabs', 'value'))
def funtion(tab):
    if tab == 'Driver Results':
        children=[
            dcc.Tab(label='Championship Podiums', value='Championship Podiums',className='custom-tab',selected_className='custom-tab--selected'),
            dcc.Tab(label='Race Podiums', value='Race Podiums',className='custom-tab',selected_className='custom-tab--selected'),
            dcc.Tab(label='Place per Season', value='Place per Season',className='custom-tab',selected_className='custom-tab--selected'),
            dcc.Tab(label='Constructors per Season', value='Constructors per Season',className='custom-tab',selected_className='custom-tab--selected'),
            dcc.Tab(label='Winners per Season', value='Winners per Season',className='custom-tab',selected_className='custom-tab--selected'),
            dcc.Tab(label='Youngest Drivers', value='Youngest Drivers',className='custom-tab',selected_className='custom-tab--selected'),
        ]
        value='Championship Podiums'

    elif tab == 'Constructor Results':
        children=[
            dcc.Tab(label='Championship Podiums', value='Championship Podiums',className='custom-tab',selected_className='custom-tab--selected'),
            dcc.Tab(label='Race Podiums', value='Race Podiums',className='custom-tab',selected_className='custom-tab--selected'),
            dcc.Tab(label='Place per Season', value='Place per Season',className='custom-tab',selected_className='custom-tab--selected'),
            dcc.Tab(label='Drivers per Season', value='Drivers per Season',className='custom-tab',selected_className='custom-tab--selected'),
            dcc.Tab(label='Winners per Season', value='Winners per Season',className='custom-tab',selected_className='custom-tab--selected'),
        ]
        value='Championship Podiums'

   # elif tab == 'Circuit Results':
   #     children=[
   #         dcc.Tab(label='Race Podiums', value='Race Podiums',className='custom-tab',selected_className='custom-tab--selected'),
   #         dcc.Tab(label='Place per Season - Driver', value='Place per Season Driver',className='custom-tab',selected_className='custom-tab--selected'),
   #         dcc.Tab(label='Place per Season - Team', value='Place per Season Team',className='custom-tab',selected_className='custom-tab--selected'),
   #         dcc.Tab(label='Winners per Season - Driver', value='Winners per Season Driver',className='custom-tab',selected_className='custom-tab--selected'),
   #         dcc.Tab(label='Winners per Season - Team', value='Winners per Season Team',className='custom-tab',selected_className='custom-tab--selected'),
   #     ]
   #     value='Race Podiums'

    elif tab == 'Nationality Results':
        children=[
            dcc.Tab(label='Championship Podiums', value='Championship Podiums',className='custom-tab',selected_className='custom-tab--selected'),
            dcc.Tab(label='Race Podiums', value='Race Podiums',className='custom-tab',selected_className='custom-tab--selected'),
        ]
        value='Championship Podiums'

    elif tab == 'Formula 1 Dashboard':
        children=[]
        value=''

    return children, value

@app.callback(
    [Output('DropdownDriver1', 'options'),
    Output('DropdownDriver2', 'options')],
    [Input('RangeSlider', 'value')])
def Dropdown1(years):
    df1=dfp8[(dfp8['raceYear']>=years[0])&(dfp8['raceYear']<=years[1])].copy()
    driverNames = [{'label': 'All', 'value': 'All'}]
    for driver in df1['driverName'].sort_values().unique():
        driverNames.append({'label':str(driver),'value':driver})
    del driverNames[0]

    return driverNames, driverNames

@app.callback(
    [Output('DropdownConstructor1', 'options'),
    Output('DropdownConstructor2', 'options')],
    [Input('RangeSlider', 'value')])
def Dropdown1(years):
    df1=dfp9[(dfp9['raceYear']>=years[0])&(dfp9['raceYear']<=years[1])].copy()
    constructorNames = [{'label': 'All', 'value': 'All'}]
    for constructor in df1['constructorName'].sort_values().unique():
        constructorNames.append({'label':str(constructor),'value':constructor})
    del constructorNames[0]

    return constructorNames, constructorNames

@app.callback(
    Output('DropdownDriverCPS', 'options'),
    [Input('RangeSliderCPS', 'value')])
def Dropdown1(years):
    df1=dfp10[(dfp10['raceYear']>=years[0])&(dfp10['raceYear']<=years[1])].copy()
    driverNames = [{'label': 'All', 'value': 'All'}]
    for driver in df1['driverName'].sort_values().unique():
        driverNames.append({'label':str(driver),'value':driver})

    return driverNames

@app.callback(
    Output('DropdownConstructorDPS', 'options'),
    [Input('RangeSlider', 'value')])
def Dropdown1(years):
    df1=dfp11[(dfp11['raceYear']>=years[0])&(dfp11['raceYear']<=years[1])].copy()
    constructorNames = [{'label': 'All', 'value': 'All'}]
    for constructor in df1['constructorName'].sort_values().unique():
        constructorNames.append({'label':str(constructor),'value':constructor})

    return constructorNames

@app.callback(
    Output('Dropdown_Nationalities', 'options'),
    [Input('Checklist', 'value'),
     Input('RangeSlider', 'value'),
     Input('mainTabs', 'value'),
     Input('secondaryTabs', 'value')])
def Dropdown_Nationalities(Checklist_options, years, tab, tab2):
    nationalities = [{'label': 'All', 'value': 'All'}]
    if tab=="Driver Results":
        if tab2=="Championship Podiums":
            df1=df0[df0['place'].isin(Checklist_options)].copy()
            df1=df1[(df1['raceYear']>=years[0])&(df1['raceYear']<=years[1])]
            nationalities = [{'label': 'All', 'value': 'All'}]
            for nationality in df1['driverNationality'].sort_values().unique():
                nationalities.append({'label':str(nationality),'value':nationality})

        if tab2=="Race Podiums":
            df1=df[(df['raceYear']>=years[0])&(df['raceYear']<=years[1])].copy()
            df1= df1[df1['positionOrder'].isin(Checklist_options)].groupby(['driverName','driverNationality','driverDob'
                                           ])['positionOrder'].count().reset_index().sort_values(
                                             by = 'positionOrder', ascending=False).rename(
                                             columns={'positionOrder': 'wins'})
            df1=df1[['driverName','driverNationality', 'driverDob','wins']]
            nationalities = [{'label': 'All', 'value': 'All'}]
            for nationality in df1['driverNationality'].sort_values().unique():
                nationalities.append({'label':str(nationality),'value':nationality})

    if tab=="Constructor Results":
        if tab2=="Championship Podiums":
            df1=df0[df0['place'].isin(Checklist_options)].copy()
            df1=df1[(df1['raceYear']>=years[0])&(df1['raceYear']<=years[1])]
            nationalities = [{'label': 'All', 'value': 'All'}]
            for nationality in df1['constructorNationality'].sort_values().unique():
                nationalities.append({'label':str(nationality),'value':nationality})

        if tab2=="Race Podiums":
            df1=df[(df['raceYear']>=years[0])&(df['raceYear']<=years[1])].copy()
            df1= df1[df1['positionOrder'].isin(Checklist_options)].groupby(['constructorName','constructorNationality'
                                           ])['positionOrder'].count().reset_index().sort_values(
                                             by = 'positionOrder', ascending=False).rename(
                                             columns={'positionOrder': 'wins'})
            df1=df1[['constructorName','constructorNationality', 'wins']]
            nationalities = [{'label': 'All', 'value': 'All'}]
            for nationality in df1['constructorNationality'].sort_values().unique():
                nationalities.append({'label':str(nationality),'value':nationality})

    return nationalities

@app.callback(
    Output('Dropdown_Circuits', 'options'),
    [Input('Checklist', 'value'),
     Input('RangeSlider', 'value'),
     Input('mainTabs', 'value'),
     Input('secondaryTabs', 'value')])
def Dropdown_Circuits(Checklist_options, years, tab, tab2):
    circuits = [{'label': 'All', 'value': 'All'}]
    if tab=="Driver Results":
        if tab2=="Race Podiums":
            df1=df[(df['raceYear']>=years[0])&(df['raceYear']<=years[1])].copy()
            circuits = [{'label': 'All', 'value': 'All'}]
            for circuit in df1['raceName'].sort_values().unique():
                circuits.append({'label':str(circuit),'value':circuit})

    return circuits

@app.callback(
     [Output('filters_podiums', 'style'),
      Output('filters_placePerSeasonDrivers', 'style'),
      Output('filters_placePerSeasonConstructors', 'style'),
      Output('filters_constructorsPerSeason', 'style'),
      Output('filters_driversPerSeason', 'style'),
      Output("filters_nationality", 'style'),
      Output("filters_circuit", 'style'),
      Output("tables", 'style'),
      Output('RangeSliderDiv', 'style'),
     ],
     [Input('mainTabs', 'value'),
      Input('secondaryTabs', 'value')])
def funtion(tab, tab2):
    stylePodiums= stylePPSD= stylePPSC=styleCPS=styleDPS =styleTables = {'width': '99%', 'display': 'none'}
    styleRangeSliderDiv={'width': '99%', 'display': 'inline-block'}
    styleNat={'width': '30%', 'display': 'inline-block'}
    styleCir={'width': '36%', 'display': 'inline-block'}

    if tab == 'Driver Results':
        if tab2 == 'Championship Podiums':
            stylePodiums= {'width': '99%', 'display': 'inline-block'}
            styleCir={'width': '36%', 'display': 'None'}
        elif tab2 == 'Race Podiums':
            stylePodiums= {'width': '99%', 'display': 'inline-block'}
        elif tab2 == 'Place per Season':
            stylePPSD= {'width': '99%', 'display': 'inline-block'}
            styleCir={'width': '36%', 'display': 'inline-block'}
        elif tab2 == 'Constructors per Season':
            styleCPS= {'width': '99%', 'display': 'inline-block'}
            styleRangeSliderDiv={'width': '99%', 'display': 'None'}
    elif tab == 'Constructor Results':
        if tab2 == 'Championship Podiums':
            stylePodiums= {'width': '99%', 'display': 'inline-block'}
            styleCir={'width': '36%', 'display': 'None'}
        elif tab2 == 'Race Podiums':
            stylePodiums= {'width': '99%', 'display': 'inline-block'}
        elif tab2 == 'Place per Season':
            stylePPSC= {'width': '99%', 'display': 'inline-block'}
        elif tab2 == 'Drivers per Season':
            styleDPS= {'width': '99%', 'display': 'inline-block'}
            styleRangeSliderDiv={'width': '99%', 'display': 'None'}
    elif tab == 'Nationality Results':
        styleNat={'width': '33%', 'display': 'None'}
        stylePodiums= {'width': '99%', 'display': 'inline-block'}
        if tab2 == 'Championship Podiums':
            styleCir={'width': '36%', 'display': 'None'}
            
        
    elif tab == 'Formula 1 Dashboard':
        stylePodiums= {'width': '99%', 'display': 'none'}
        styleRangeSliderDiv={'width': '99%', 'display': 'None'}
        styleTables = {'width': '100%','display': 'inline-block'}

    return stylePodiums, stylePPSD, stylePPSC, styleCPS, styleDPS, styleNat, styleCir, styleTables, styleRangeSliderDiv


@app.callback(
    [Output('Dropdown3', 'options'),
     Output('Dropdown3', 'value'),
     Output('Dropdown3', 'style')],
    [Input('Dropdown1', 'value'),
     Input('Dropdown2', 'value'),
     Input('Dropdown3', 'value')])
def funtion(year, topic, subtopic):
    value="All"
    style={'font-size': '14px', 'color':'#808080','font-family':"system-ui",'display': 'block'}
    df1=df.copy()
    df1=df1[df1['raceYear']==year]
    if topic == "RACES":
        if len(df1[df1["raceName"]==subtopic])>0:
            value = subtopic
        options=np.array(df1.groupby(['raceRound','raceName'])['points'].sum().reset_index()['raceName'])
        dictionary = [{'label': 'All', 'value': 'All'}]
        for item in options:
            dictionary.append({'label':str(item),'value':item})
        return dictionary, value, style

    if topic == "DRIVERS":
        if len(df1[df1["driverName"]==subtopic])>0:
            value = subtopic
        options=df1['driverName'].sort_values().unique()
        dictionary = [{'label': 'All', 'value': 'All'}]
        for item in options:
            dictionary.append({'label':str(item),'value':item})
        return dictionary, value, style

    if topic == "TEAMS":
        if len(df1[df1["constructorName"]==subtopic])>0:
            value = subtopic
        options=df1['constructorName'].sort_values().unique()
        dictionary = [{'label': 'All', 'value': 'All'}]
        for item in options:
            dictionary.append({'label':str(item),'value':item})
        return dictionary, value, style

    if topic == "FASTEST LAP":
        style={'display': 'none'}
        return years, value, style
    
if __name__ == '__main__':
    app.run_server(debug=True, port=8078)